In [1]:
import warnings

warnings.filterwarnings("ignore")

import copy
import json
import os
import re
import sys
from collections import Counter
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rich import print
from scipy.ndimage import convolve
from tqdm import tqdm

In [2]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """ Convert a list of string to a list of other types
    
    pattern: a list of types for one item. 
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [3]:
%%time
p1_start = 6
p2_start = 9

die = np.array([np.arange(1, 100)] * 10).flatten()
die_pos = 0
p1_pos = p1_start
p2_pos = p2_start
p1_score = 0
p2_score = 0

# After every 20 rounds, the game returns to the initial state

for i in range(20):
    add_num = np.sum(die[die_pos : die_pos + 3]) % 10
    if i % 2:
        p2_pos = (p2_pos + add_num) % 10
        p2_score += p2_pos if p2_pos != 0 else 10
    else:
        p1_pos = (p1_pos + add_num) % 10
        p1_score += p1_pos if p1_pos != 0 else 10
    die_pos += 3

final_score = 1000
base_round = int(final_score / max(p1_score, p2_score))

base_score_p1 = p1_score * base_round
base_score_p2 = p2_score * base_round
# print(base_round, base_score_p1, base_score_p2)

# die = np.array([np.arange(1, 100)] * 10).flatten()
# die_pos = 0
p1_pos = p1_start
p2_pos = p2_start
p1_score = base_score_p1
p2_score = base_score_p2

for i in range(20):
    add_num = np.sum(die[die_pos : die_pos + 3]) % 10
    if i % 2:
        p2_pos = (p2_pos + add_num) % 10
        p2_score += p2_pos if p2_pos != 0 else 10
    else:
        p1_pos = (p1_pos + add_num) % 10
        p1_score += p1_pos if p1_pos != 0 else 10
    die_pos += 3
    if (p1_score >= final_score) or (p2_score >= final_score):
        # print(i + 1, p1_score, p2_score)
        break

print(f"Answer to Q1: {p2_score * (i + 1 + base_round * 20) * 3}")

Answer to Q1: 943263

Wall time: 95 ms


In [4]:
%%time
# create 3**3=27 univ every time
add_dict = {}
for i in range(1, 4):
    for j in range(1, 4):
        for k in range(1, 4):
            key = i + j + k
            if key in add_dict:
                add_dict[key] += 1
            else:
                add_dict[key] = 1

# Initiate all the possible univ state and their counts
universe_count = {}
# Key: player1_pos, player2_pos, player1_score, player2_score, next_player
# Value: count
for i in range(10):
    for j in range(10):
        for k in range(21):
            for l in range(21):
                for m in range(1, 3):
                    universe_count[(i, j, k, l, m)] = 0

# Only 1 univ at the beginning
universe_count[(6, 9, 0, 0, 1)] = 1

player1_win = 0
player2_win = 0

# Continue when there are more than 1 unfinished univ
while np.sum(list(universe_count.values())) > 0:
    for (
        (player1_pos, player2_pos, player1_score, player2_score, next_player),
        count,
    ) in universe_count.items():
        if count == 0:
            continue
        if next_player == 2:
            for add_num, uni_count in add_dict.items():
                new_player2_pos = (player2_pos + add_num) % 10
                new_player2_score = player2_score + (
                    new_player2_pos if new_player2_pos != 0 else 10
                )
                if new_player2_score >= 21:
                    player2_win += count * uni_count
                else:
                    # Jump to next univ
                    universe_count[
                        (
                            player1_pos,
                            new_player2_pos,
                            player1_score,
                            new_player2_score,
                            1,
                        )
                    ] += (count * uni_count)
        elif next_player == 1:
            for add_num, uni_count in add_dict.items():
                new_player1_pos = (player1_pos + add_num) % 10
                new_player1_score = player1_score + (
                    new_player1_pos if new_player1_pos != 0 else 10
                )
                if new_player1_score >= 21:
                    player1_win += count * uni_count
                else:
                    # Jump to next univ
                    universe_count[
                        (
                            new_player1_pos,
                            player2_pos,
                            new_player1_score,
                            player2_score,
                            2,
                        )
                    ] += (count * uni_count)

        # Reset the start univ num to 0
        universe_count[
            (player1_pos, player2_pos, player1_score, player2_score, next_player)
        ] = 0

print(f"Answer to Q2: {max(player1_win, player2_win)}")

Answer to Q2: 486638407378784

Wall time: 754 ms
